# Portofolio Optimization

In [9]:
import yfinance as yf
import pandas as pd
import datetime
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize

Section 1: Define Tickers and Time Range

In [96]:
tickers = ['ASII', 'ESSA', 'BBCA', 'INCO', 'INDF','CTRA']
end_date = datetime.today()
start_date = end_date - timedelta(days = 5*365)

#First, we define a list of tikers for the assets we want to include in the portfolio. In this example, we use several stocks representing various sector classess. 
##Then, we set the start and end dates for our analysis. 
###We use a five-year historical time range for our calculations

Section 2: Download Adjusted Close Prices

In [97]:
adj_close_df = pd.DataFrame()  #create an empty DataFrame to store the adjusted close prices of each asset. 

for ticker in tickers:
    data = yf.download(ticker, start = start_date,end = end_date)
    adj_close_df[ticker] = data['Adj Close']
#We use the YFinance library to download the data from Yahoo Finance

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [106]:
print(adj_close_df)

              ASII       ESSA       BBCA       INCO       INDF       CTRA
Date                                                                     
2018-08-27  0.5950  13.969443  44.322708  40.528717        NaN  19.113424
2018-08-28  0.5000  13.908967  44.093136  40.360199        NaN  19.113424
2018-08-29  0.5000  13.908967  44.322708  40.250664        NaN  19.225018
2018-08-30  0.4500  13.831217  44.093136  40.023163        NaN  19.248930
2018-08-31  0.4200  13.891690  43.581051  40.360199        NaN  18.993868
...            ...        ...        ...        ...        ...        ...
2023-08-17  0.0006  16.510000  59.090000  51.209999  34.020000  27.930000
2023-08-18  0.0006  16.690001  59.049999  51.349998  34.006001  27.780001
2023-08-21  0.0006  16.770000  58.970001  51.650002  34.242001  28.200001
2023-08-22  0.0006  16.219999  58.639999  51.939999  34.108002  27.900000
2023-08-23  0.0006  16.280001  59.259998  52.490002  34.896000  27.629999

[1256 rows x 6 columns]


Section 3: Calculate Lognormal Returns

In [99]:
#use lognormal returns for each tickers because it is a lot easier for calculate future returns

log_returns = np.log(adj_close_df / adj_close_df.shift(1)) 
#one day price divided with the previous day price
log_returns = log_returns.dropna() 
#drop any missing values

Section 4: Calculate Covariance Matrix

In [100]:
#The covariance matrix is the most importance of all of here. This is how we measure the total risk of portfolio
##Each of these asset have a certain correlation and covariance with each other asset in portfolio
###so we made these matrix in order to standard deviation or risk in the most acknowledge possible

cov_matrix = log_returns.cov()*252 
#we multiply with 252 because we want to see daily return but with the annualized log returns

print(cov_matrix)

          ASII      ESSA      BBCA      INCO      INDF      CTRA
ASII  6.637530  0.028331  0.028713  0.024117  0.026431  0.009116
ESSA  0.028331  0.073766  0.010256  0.006022  0.010485  0.011074
BBCA  0.028713  0.010256  0.032249  0.016519  0.021079  0.033268
INCO  0.024117  0.006022  0.016519  0.035000  0.030107  0.010921
INDF  0.026431  0.010485  0.021079  0.030107  0.046702  0.014475
CTRA  0.009116  0.011074  0.033268  0.010921  0.014475  0.163006


Section 5: Define Portfolio Performance Matrix

In [101]:
#this line of code calculates the portfolio variabce, which is a measure of the risk associated with a portfolio of assets. 
##It represents the combines volatility of the assets in the portfolio, taking into account their individual volatilities and correlations with each other

def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)
    
    #standard deviation is a square root of it, right?



5.2 Calculate the expected return

In [102]:
# Key assumption: Expected return are based on historical returns
## average of past return
#### for CFA holders, they might able to see the alpha (expected return). 
##### While for us, this might the best option

def expected_return(weights, log_returns):
    return np.sum(log_returns.mean(*weights)*252)


5.3 Calculate Sharpe Ratio and the function to minimze (sharpe ratio)

In [129]:
#The key formula to measure Sharpe Ratio is by:
#Sharpe Ratio = (Portfolio Return - Risk Free Rate)/St.Dev

def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(Weights, cov_matrix)

Section 6: Portfolio Optimization

6.1 Set the risk-free rate

In [104]:
#We determine risk rate by used Bank Indonesia 7Day Revere-Repo (BI7DR)
##In which, per 24 August 2023, BI-7 Day RR set as 5,75%
###Therefore, the number itself .0575

# "https://www.bi.go.id/id/statistik/indikator/bi-7day-rr.aspx"
    
risk_free_rate = .0575

6.1 Define the function to minimize (negative sharpe ratio)

In [ ]:
#In the case of the scipy.optimize.minimze() function, there is no direct method to find the maximum value of a function
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

6.2 Set the constraints and bounds

In [123]:
#constrains are conditions that must be met by the solutions during the optimization process.
##In this case, the constraint is that the sum of all portfolio weights must be equal to 1
###The constraint variable is a dictionary with two keys: 'type' and 'fun'

####'type' is set to 'eq' which means 'equally constraint'
#### and 'fun' is assigned the function check_sum, which checks if the sum of the portfolio weights equals 1

# Bounds are the limits placed on the variables during the optimization process
## In this case, the variables are the portfolio weights, and each weight should be between 0 and 1

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1} 
#we are basically make to measure that the weights in the portfolio sum to 1

bounds = [(0, 0.5) for _ in range(len(tickers))] 
# Pretty important. 0 is the lower bounds. It means we cannot go short in these asset or sell asset that we do not know
# 0.5 upper bounds, we cant have more than any 50 percent of our portfolio in any of these single securities
##sometimes we see in NASDAQ for example, or in NIKKEI, that some stocks are hike and outperformed else, so the portfolio optimization became innacurate kinda 'huh i should go for 80 percent increases' which is not true


6.3 Set the initial weights

In [124]:
initial_weights = np.array([1/len(tickers)]*len(tickers))

print(initial_weights)

[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


6.4 Optimize the weights to maximize Sharpe Ratio, then geh the optimal weights

In [130]:
optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)
#'SLSQP' stands for Sequential Least Squares Quadratic Programming, which is a numerical optimization techniques suitable for solving nonlinear optimization problem with constraints


optimal_weights = optimized_results.x

TypeError: NDFrame._add_numeric_operations.<locals>.mean() takes from 1 to 5 positional arguments but 7 were given

Section 7: Analyze the Optimal Portfolio

7.1 Display analytics of the optimal portfolio

In [127]:
print("Optimal Weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")

Optimal Weights:


NameError: name 'optimal_weights' is not defined

7.2 Display the final portfolio in a plot

In [128]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(tickers, optimal_weights)

plt.xlabel('Assets')
plt.ylabel('Optimal Weights')
plt.title('Optimal Portfolio Weights')

plt.show()

NameError: name 'optimal_weights' is not defined

<Figure size 1000x600 with 0 Axes>